In [ ]:
import pandas as pd
import numpy as np

# Simulated data
np.random.seed(42)
data = {
    'vehicle_count': np.random.randint(0, 100, 100),
    'queue_length': np.random.randint(0, 300, 100),
    'density': np.random.uniform(0.05, 0.8, 100),
    'time_of_day': np.random.choice(['morning', 'afternoon', 'evening', 'night'], 100),
    'congestion_level': np.random.choice(['low', 'medium', 'high'], 100)  # Replace with logic-based labels
}

df = pd.DataFrame(data)
df.head()



,vehicle_count,queue_length,density,time_of_day,congestion_level
0,51,216,0.507173,morning,low
1,92,251,0.427009,afternoon,low
2,14,187,0.088609,morning,high
3,71,40,0.258985,evening,medium
4,60,156,0.731199,evening,low


In [ ]:
import cv2
import numpy as np
import torch
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt

# Load the YOLOv5 model from PyTorch Hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
results = model('/content/fc road.jpg')  # YOLO detections
detections = results.pandas().xyxy[0]
vehicle_classes = ['car', 'truck', 'bus', 'motorcycle']
vehicle_detections = detections[detections['name'].isin(vehicle_classes)]
vehicle_count = len(vehicle_detections)
print(vehicle_count)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-31 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


12


In [ ]:
stop_line_y = 500  # Manually defined based on image/video
max_y = max(row['ymax'] for _, row in vehicle_detections.iterrows())
queue_length = max_y - stop_line_y if max_y > stop_line_y else 0
print(queue_length)

350.05426025390625


In [ ]:
grid_size = 5
height, width = 640, 640  # Image dimensions
grid_counts = np.zeros((grid_size, grid_size))
grid_height, grid_width = height // grid_size, width // grid_size

for _, row in vehicle_detections.iterrows():
    x_center = (row['xmin'] + row['xmax']) / 2
    y_center = (row['ymin'] + row['ymax']) / 2
    grid_x = int(x_center // grid_width)
    grid_y = int(y_center // grid_height)
    if 0 <= grid_x < grid_size and 0 <= grid_y < grid_size:
        grid_counts[grid_y, grid_x] += 1

density = np.mean(grid_counts)  # Average density across the grid
print(density)

0.0


In [ ]:
def label_congestion(vehicle_count, queue_length, density):
    if vehicle_count < 10 and queue_length < 50 and density < 0.1:
        return 'low'
    elif 10 <= vehicle_count < 30 and queue_length < 150:
        return 'medium'
    else:
        return 'high'

df['congestion_level'] = df.apply(
    lambda row: label_congestion(row['vehicle_count'], row['queue_length'], row['density']),
    axis=1
)
print(df)

    vehicle_count  queue_length   density time_of_day congestion_level
0              51           216  0.507173     morning             high
1              92           251  0.427009   afternoon             high
2              14           187  0.088609     morning             high
3              71            40  0.258985     evening             high
4              60           156  0.731199     evening             high
..            ...           ...       ...         ...              ...
95             84           112  0.765446     evening             high
96             79           100  0.736148       night             high
97             81           112  0.327619     morning             high
98             52            80  0.061592     morning             high
99             23           186  0.746239       night             high

[100 rows x 5 columns]


In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
time_encoded = encoder.fit_transform(df[['time_of_day']]).toarray()
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names_out(['time_of_day']))
features = pd.concat([df[['vehicle_count', 'queue_length', 'density']], time_encoded_df], axis=1)
labels = df['congestion_level']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model_rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9
Classification Report:
               precision    recall  f1-score   support

        high       0.90      1.00      0.95        18
      medium       0.00      0.00      0.00         2

    accuracy                           0.90        20
   macro avg       0.45      0.50      0.47        20
weighted avg       0.81      0.90      0.85        20



In [ ]:
import joblib

joblib.dump((model_rf, encoder), 'random_forest_congestion_model.pkl')


['random_forest_congestion_model.pkl']

In [ ]:
def predict_congestion(vehicle_count, queue_length, density, time_of_day):
    # Load the model and encoder
    model_rf, encoder = joblib.load('random_forest_congestion_model.pkl')

    # Prepare input features
    input_data = pd.DataFrame({
        'vehicle_count': [vehicle_count],
        'queue_length': [queue_length],
        'density': [density],
        'time_of_day': [time_of_day]
    })
    time_encoded = encoder.transform(input_data[['time_of_day']]).toarray()
    time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names_out(['time_of_day']))
    input_features = pd.concat([input_data[['vehicle_count', 'queue_length', 'density']], time_encoded_df], axis=1)

    # Predict congestion level
    prediction = model_rf.predict(input_features)
    return prediction[0]

# Example prediction
print(predict_congestion(vehicle_count=25, queue_length=80, density=0.3, time_of_day='evening'))


high
